In [1]:
import numpy as np
import pandas as pd
from os import listdir

import plotly.plotly as py
py.sign_in('lemoga', 'tljxTgI7TUZjMSUOtbsP')
import plotly
import plotly.graph_objs as go
plotly.offline.init_notebook_mode() # run at the start of every notebook

ModuleNotFoundError: No module named 'plotly'

In [10]:
#Read lengths file
referenceLengths =  pd.read_csv("0.bacterialGenomes/BacterialGenomes.length.txt",sep='\t',header=None)
referenceLengths.columns = ["REFERENCE","LENGTH"]
referenceLengths.set_index("REFERENCE",inplace=True)
referenceLengths.sort_values(by="LENGTH",inplace=True)
referenceLengths.head()

,LENGTH
REFERENCE,
MECZ01000002.1,21881
CP000452.1,55635
CM003440.1,61247
CP017830.1,91776
CP013211.1,112031


In [23]:
def logNormalization(covMatrix,referenceLengths):
    referenceLengthsSample = referenceLengths.loc[covMatrix.index]
    referenceLengthsSample.sort_values(by="LENGTH",inplace=True)
    covMatrix = covMatrix.loc[referenceLengthsSample.index]
    #Normalize abundance matrix and put nan values to bins greater than contig size
    binSize = 100000
    for index,row in covMatrix.iterrows():
        covMatrix.loc[index][(referenceLengths.loc[index]["LENGTH"]/np.int(binSize))+1:] = np.NAN
    covMatrixNormalized = np.log10(covMatrix.add(1,axis=1))
    return covMatrixNormalized

In [12]:
#create heatmaps
def makePlot(covMatrixNormalized,name):
    data = [go.Heatmap(z=covMatrixNormalized.values.tolist(),
                       y=covMatrixNormalized.index, x=covMatrixNormalized.columns.values,
                       colorscale="Hot")]

    layout = go.Layout(title=name, width=1000,height=1000,autosize=False)
    fig = go.Figure(data=data, layout=layout)

    # Adjust the existing axes
    layout['xaxis'].update(domain=[0, 0.7])  # decrease domain of x-axis1 (norm. coord.)

    # Set up new axes
    layout.update(xaxis2=dict(
        domain=[0.75, 1], # domain of x-axis2
        zeroline=False,   # remove x=0 line
        showgrid=True,     # show horizontal grid line
        title="Median Coverage"
    ))

    median = covMatrixNormalized.median(axis=1)
    trace2 = go.Scatter(x=median,y=covMatrixNormalized.index,xaxis="x2",marker=dict(color="blue"))
    data.extend([trace2])

    fig = go.Figure(data=data, layout=layout)
    return fig
    #plotly.offline.iplot(fig)
    #plotly.plotly.image.save_as(fig, 'K2.contaminants.pdf')

In [24]:
figures = []
folder = "10.afterDepletion/coverageMatrices/"
#folder = "7.coverageMatrices.sorted.bacteria/"
for matrix in listdir(folder):
    if ".matrix.csv" in matrix:
        #makeThePlot
        covMatrix = pd.read_csv(folder+matrix,header=0,index_col=0,sep='\t')
        covMatrixNormalized = logNormalization(covMatrix,referenceLengths)
        figures.append(makePlot(covMatrixNormalized,matrix.split('.')[0]))
        #Save Descriptive stats
        stats = pd.DataFrame([covMatrixNormalized.median(axis=1),covMatrixNormalized.mean(axis=1)],index=["Median","Mean"])
        stats = stats.T
        stats.to_csv(matrix.split('.')[0]+".stats.csv",header=True,index=True,sep='\t')

In [27]:
for i in range(37,len(figures)):
    plotly.plotly.image.save_as(figures[i],figures[i]["layout"]['title']+'.pdf')
    print i

37
38
39
40
41
42
43
44
45
46
47


In [28]:
%%bash
mkdir 10.afterDepletion/plots
#mv *.pdf 7.coverageMatrices.sorted.bacteria/
mv *.pdf 10.afterDepletion/plots
#mv *.stats.csv 7.coverageMatrices.sorted.bacteria/
mv *.stats.csv 10.afterDepletion/plots

mkdir: cannot create directory ‘10.afterDepletion/plots’: File exists
